# Import libraries

In [1]:
import cnn_models
import data_preparation
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.svm import SVC
import tensorflow as tf
import data_augmentation
import random

/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kappa/opt/miniconda3/envs/dsim

# Fix seed

In [2]:
SEED = 10
random.seed(SEED)
tf.random.set_random_seed(SEED)

# Load recordings
## STANDARD RECORDINGS - No spectrogram normalization

In [3]:
recordings = data_preparation.load_recordings(paths=['recordings', 'output'])

Loading from recordings



Loading from output


Raw recordings have different lengths? Let's check it out:

In [4]:
min_y = min(map(np.shape, recordings))[0]
max_y = max(map(np.shape, recordings))[0]
print(min_y, max_y)

2784 50335


Yes! They vary a lot. For this reason we can add 0s at the beginning and at the end in order to uniform them

**TO DO: Another strategy may be to vary spectrogram params so that spectograms will have the same length**

In [5]:
pad_recordings = data_preparation.pad_zeros(recordings)

pad_zeros >>>
pad_zeros <<<


What is the range now?

In [6]:
min_y = min(map(np.shape, pad_recordings))[0]
max_y = max(map(np.shape, pad_recordings))[0]
print(min_y, max_y)

50335 50335


We can now compute spectograms:

In [7]:
spects = [data_preparation.compute_spectrogram(x) for x in pad_recordings]
spects = np.array(spects)

The procedure worked as expected! we can now move on to the prediction task

In [8]:
labels_speakers = data_preparation.load_labels(paths=['recordings', 'output'], label_type="speakers")
labels_digits = data_preparation.load_labels(paths=['recordings', 'output'])

Normalize spectrograms

In [9]:
norm_spects = [data_preparation.compute_spectrogram(x, normalize=True) for x in pad_recordings]
norm_spects = np.array(norm_spects)

## Augmentation

In [10]:
%%time
X_train_digit, y_train_digit, X_val_digit, y_val_digit, X_test_digit, y_test_digit = data_preparation.prepare_augmented_recordings(audio_dirs= ['output', 'recordings'],
                             y_type= ['digit', 'digit'],
                             n_category_test=15,
                             include_pitch=True)

conversion_done!
compute_spectrograms >>>
pad_zeros >>>
pad_zeros <<<
pad_zeros >>>
pad_zeros <<<
pad_zeros >>>
pad_zeros <<<
Padding done
compute_spectrograms <<<
CPU times: user 6min 4s, sys: 22.3 s, total: 6min 26s
Wall time: 5min 24s


In [11]:
print("Lengths : {}, {}, {}, {}".format(len(X_train_digit),
                                                 len(y_train_digit),
                                                 len(X_test_digit),
                                                 len(y_test_digit),))

Lengths : 18480, 18480, 300, 300


In [12]:
%%time
X_train_speaker, y_train_speaker, X_val_speaker, y_val_speaker, X_test_speaker, y_test_speaker = data_preparation.prepare_augmented_recordings(audio_dirs= ['output', 'recordings'],
                             y_type= ['speakers_us', 'speakers_default'],
                             n_category_test=30,
                             include_pitch=True)

conversion_done!
compute_spectrograms >>>
pad_zeros >>>
pad_zeros <<<
pad_zeros >>>
pad_zeros <<<
pad_zeros >>>
pad_zeros <<<
Padding done
compute_spectrograms <<<
CPU times: user 5min 28s, sys: 16.9 s, total: 5min 45s
Wall time: 4min 7s


In [13]:
print("Lengths : {}, {}, {}, {}".format(len(X_train_speaker),
                                                 len(y_train_speaker),
                                                 len(X_test_speaker),
                                                 len(y_test_speaker)))

Lengths : 19008, 19008, 240, 240


# Standard recordings
## Numbers

Split data in train, val and test

In [15]:
X_train, X_val, X_test, y_train, y_val, y_test = data_preparation.split_train_test_baseline_spectrograms(spects, labels_digits)

In [16]:
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")

In [17]:
%%time
clf1 = clf1.fit(X_train, y_train)

CPU times: user 2min 19s, sys: 1.05 s, total: 2min 20s
Wall time: 2min 28s


In [18]:
%%time
y_pred = clf1.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.20      0.32        44
           1       0.46      0.39      0.42        44
           2       0.69      0.19      0.30        48
           3       0.60      0.31      0.41        58
           4       0.22      0.46      0.29        35
           5       0.83      0.37      0.51        51
           6       0.15      0.63      0.24        54
           7       0.78      0.31      0.44        45
           8       0.88      0.27      0.41        56
           9       0.85      0.49      0.62        45

    accuracy                           0.36       480
   macro avg       0.62      0.36      0.40       480
weighted avg       0.63      0.36      0.40       480

CPU times: user 32 s, sys: 375 ms, total: 32.4 s
Wall time: 35.5 s


### Normalize spectrograms

In [19]:
X_train, X_val, X_test, y_train, y_val, y_test = data_preparation.split_train_test_baseline_spectrograms(norm_spects, labels_digits)

In [20]:
%%time
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")
clf1 = clf1.fit(X_train, y_train)

CPU times: user 1min 54s, sys: 2.16 s, total: 1min 56s
Wall time: 2min 20s


In [21]:
%%time
y_pred = clf1.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.68      0.75        44
           1       0.68      0.73      0.70        44
           2       0.45      0.83      0.59        48
           3       0.70      0.28      0.40        58
           4       0.90      0.80      0.85        35
           5       0.84      0.73      0.78        51
           6       0.39      0.61      0.48        54
           7       0.69      0.69      0.69        45
           8       0.75      0.71      0.73        56
           9       0.93      0.60      0.73        45

    accuracy                           0.65       480
   macro avg       0.72      0.67      0.67       480
weighted avg       0.71      0.65      0.66       480

CPU times: user 31.3 s, sys: 539 ms, total: 31.9 s
Wall time: 33.1 s


### CNNs

#### Normalized spectrograms

In [22]:
X_train, X_val, X_test, y_train, y_val, y_test, input_shape = data_preparation.split_train_test_nn(norm_spects, labels_digits)

In [23]:
model = cnn_models.paper_architecture(10, input_shape=input_shape)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6528)              0         
_________________________________________________________________
dense (Dense)        

In [27]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=3)

In [25]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val, y_val))

Train on 1440 samples, validate on 480 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1440/1440 [==============================] - 18s 12ms/sample - loss: 2.2765 - acc: 0.1444 - val_loss: 2.2258 - val_acc: 0.2583
Epoch 2/10
1440/1440 [==============================] - 26s 18ms/sample - loss: 2.1792 - acc: 0.2361 - val_loss: 2.0557 - val_acc: 0.4083
Epoch 3/10
1440/1440 [==============================] - 24s 17ms/sample - loss: 2.0274 - acc: 0.2917 - val_loss: 1.8084 - val_acc: 0.4417
Epoch 4/10
1440/1440 [==============================] - 17s 12ms/sample - loss: 1.8210 - acc: 0.3597 - val_loss: 1.5928 - val_acc: 0.4313
Epoch 5/10
1440/1440 [==============================] - 18s 12ms/sample - loss: 1.6220 - acc: 0.4111 - val_loss: 1.4830 - val_acc: 0.4500
Epoch 6/10
1440/1440 [==============================] - 21s 14ms/sample - loss: 1.4927 - acc: 0.4715 - val_loss: 1.3169 - val_acc: 0.5188
Epoch 7/10
1440/1440 [==============================] - 16s 11ms/sample - los

In [26]:
y_val_nn = np.argmax(y_val, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_val)
print(classification_report(y_val_nn, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.84      0.76        44
           1       0.81      0.77      0.79        44
           2       0.62      0.85      0.72        48
           3       0.67      0.50      0.57        58
           4       0.91      0.86      0.88        35
           5       0.71      0.78      0.75        51
           6       0.56      0.72      0.63        54
           7       0.82      0.51      0.63        45
           8       0.64      0.70      0.67        56
           9       0.93      0.58      0.71        45

    accuracy                           0.70       480
   macro avg       0.74      0.71      0.71       480
weighted avg       0.73      0.70      0.70       480



#### Standard spectrogram

In [27]:
X_train, X_val, X_test, y_train, y_val, y_test, input_shape = data_preparation.split_train_test_nn(spects, labels_digits)

In [28]:
model = cnn_models.paper_architecture(10, input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6528)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               652900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
__________

In [29]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val, y_val))

Train on 1440 samples, validate on 480 samples
Epoch 1/10
1440/1440 [==============================] - 17s 11ms/sample - loss: 2.3281 - acc: 0.1465 - val_loss: 2.1928 - val_acc: 0.2229
Epoch 2/10
1440/1440 [==============================] - 19s 13ms/sample - loss: 2.2335 - acc: 0.2097 - val_loss: 2.1541 - val_acc: 0.2562
Epoch 3/10
1440/1440 [==============================] - 19s 13ms/sample - loss: 2.1551 - acc: 0.2354 - val_loss: 2.1004 - val_acc: 0.2979
Epoch 4/10
1440/1440 [==============================] - 21s 14ms/sample - loss: 2.1135 - acc: 0.2708 - val_loss: 2.0296 - val_acc: 0.3083
Epoch 5/10
1440/1440 [==============================] - 23s 16ms/sample - loss: 2.0584 - acc: 0.3049 - val_loss: 1.9638 - val_acc: 0.3729
Epoch 6/10
1440/1440 [==============================] - 19s 13ms/sample - loss: 2.0141 - acc: 0.3160 - val_loss: 1.9351 - val_acc: 0.3729
Epoch 7/10
1440/1440 [==============================] - 22s 16ms/sample - loss: 1.9814 - acc: 0.3250 - val_loss: 1.9246 - val

In [30]:
y_val_nn = np.argmax(y_val, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_val)
print(classification_report(y_val_nn, y_pred))

              precision    recall  f1-score   support

           0       0.53      0.39      0.45        44
           1       0.62      0.45      0.53        44
           2       0.34      0.40      0.37        48
           3       0.26      0.48      0.34        58
           4       0.55      0.17      0.26        35
           5       0.74      0.39      0.51        51
           6       0.90      0.17      0.28        54
           7       0.50      0.67      0.57        45
           8       0.53      0.30      0.39        56
           9       0.35      0.89      0.50        45

    accuracy                           0.43       480
   macro avg       0.53      0.43      0.42       480
weighted avg       0.53      0.43      0.42       480



From what we can see normalising spectrograms is the way to go. Let's use it by default

## TO DO : Pick best model, train it on X_train + X_val, evaluate on X_test

## Speakers
### SVD

In [31]:
X_train, X_val, X_test, y_train, y_val, y_test = data_preparation.split_train_test_baseline_spectrograms(norm_spects, labels_speakers)

In [32]:
%%time
clf1 = SVC(kernel='rbf', class_weight='balanced', gamma="auto")
clf1 = clf1.fit(X_train, y_train)

CPU times: user 1min 23s, sys: 1.72 s, total: 1min 25s
Wall time: 1min 33s


In [33]:
%%time
y_pred = clf1.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         ale       0.87      0.95      0.91        21
      alinda       0.92      1.00      0.96        12
        gian       0.90      1.00      0.95        19
     jackson       0.94      0.94      0.94        89
      khaled       0.77      0.91      0.83        22
     nicolas       0.96      0.92      0.94       101
        theo       0.77      0.82      0.79       112
    yweweler       0.86      0.75      0.80       104

    accuracy                           0.87       480
   macro avg       0.87      0.91      0.89       480
weighted avg       0.87      0.87      0.87       480

CPU times: user 29.4 s, sys: 584 ms, total: 30 s
Wall time: 33.9 s


### CNN

For neural networks it is not possible to pass the labels as-is: we need to transform them in numbers. The safest way is through one-hot encoding

In [34]:
y, target_names = data_preparation.transform_categorical_y(labels_speakers)

In [35]:
X_train, X_val, X_test, y_train, y_val, y_test, input_shape = data_preparation.split_train_test_nn(norm_spects, y, number_mode=False)

In [36]:
model = cnn_models.paper_architecture(8, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6528)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 80)                522320    
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0         
__________

In [37]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val, y_val))

Train on 1440 samples, validate on 480 samples
Epoch 1/10
1440/1440 [==============================] - 21s 15ms/sample - loss: 1.9242 - acc: 0.2444 - val_loss: 1.8312 - val_acc: 0.1896
Epoch 2/10
1440/1440 [==============================] - 19s 13ms/sample - loss: 1.7932 - acc: 0.3368 - val_loss: 1.6464 - val_acc: 0.3812
Epoch 3/10
1440/1440 [==============================] - 20s 14ms/sample - loss: 1.6188 - acc: 0.4611 - val_loss: 1.4285 - val_acc: 0.5938
Epoch 4/10
1440/1440 [==============================] - 19s 13ms/sample - loss: 1.4634 - acc: 0.5250 - val_loss: 1.2005 - val_acc: 0.6146
Epoch 5/10
1440/1440 [==============================] - 19s 13ms/sample - loss: 1.3037 - acc: 0.5667 - val_loss: 1.0257 - val_acc: 0.6729
Epoch 6/10
1440/1440 [==============================] - 18s 13ms/sample - loss: 1.1794 - acc: 0.6056 - val_loss: 0.9399 - val_acc: 0.6854
Epoch 7/10
1440/1440 [==============================] - 20s 14ms/sample - loss: 1.0833 - acc: 0.6361 - val_loss: 0.8572 - val

In [38]:
Y_val_nn = np.argmax(y_val, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_val)
print(classification_report(Y_val_nn, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.62      0.76        21
           1       0.55      0.50      0.52        12
           2       0.75      0.32      0.44        19
           3       0.76      0.88      0.82        89
           4       0.56      0.23      0.32        22
           5       0.78      0.96      0.86       101
           6       0.91      0.69      0.78       112
           7       0.76      0.92      0.83       104

    accuracy                           0.79       480
   macro avg       0.76      0.64      0.67       480
weighted avg       0.79      0.79      0.77       480



#### Paper - batch_normalisation=True

In [39]:
model = cnn_models.paper_architecture(8, input_shape, batch_normalisation=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 63, 156, 32)       544       
_________________________________________________________________
batch_normalization_v1 (Batc (None, 63, 156, 32)       128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 77, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 37, 64)        32832     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 14, 37, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 17, 64)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6528)              0         
__________

In [40]:
%%time
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val, y_val))

Train on 1440 samples, validate on 480 samples
Epoch 1/10
1440/1440 [==============================] - 43s 30ms/sample - loss: 1.3107 - acc: 0.5917 - val_loss: 1.7392 - val_acc: 0.3438
Epoch 2/10
1440/1440 [==============================] - 43s 30ms/sample - loss: 0.7045 - acc: 0.7882 - val_loss: 1.8618 - val_acc: 0.2333
Epoch 3/10
1440/1440 [==============================] - 41s 29ms/sample - loss: 0.4702 - acc: 0.8708 - val_loss: 2.0463 - val_acc: 0.2333
Epoch 4/10
1440/1440 [==============================] - 39s 27ms/sample - loss: 0.3931 - acc: 0.8972 - val_loss: 1.8883 - val_acc: 0.2333
CPU times: user 5min 47s, sys: 1min 5s, total: 6min 52s
Wall time: 2min 49s


In [41]:
y_pred = model.predict_classes(X_val)
print(classification_report(Y_val_nn, y_pred))

              precision    recall  f1-score   support

           0       0.05      0.19      0.09        21
           1       0.00      0.00      0.00        12
           2       0.00      0.00      0.00        19
           3       0.95      0.24      0.38        89
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00       101
           6       0.38      0.94      0.54       112
           7       0.32      0.34      0.33       104

    accuracy                           0.34       480
   macro avg       0.21      0.21      0.17       480
weighted avg       0.34      0.34      0.27       480



/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Data augmentation
## Speaker

In [14]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_train_speaker = enc.fit_transform(np.array(y_train_speaker).reshape(-1, 1)).toarray()
y_val_speaker = enc.transform(np.array(y_val_speaker).reshape(-1, 1)).toarray()
y_test_speaker = enc.transform(np.array(y_test_speaker).reshape(-1, 1)).toarray()
label_0 = enc.inverse_transform(np.array([1, 0, 0, 0, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_1 = enc.inverse_transform(np.array([0, 1, 0, 0, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_2 = enc.inverse_transform(np.array([0, 0, 1, 0, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_3 = enc.inverse_transform(np.array([0, 0, 0, 1, 0, 0, 0, 0]).reshape(1, -1))[0][0]
label_4 = enc.inverse_transform(np.array([0, 0, 0, 0, 1, 0, 0, 0]).reshape(1, -1))[0][0]
label_5 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 1, 0, 0]).reshape(1, -1))[0][0]
label_6 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 0, 1, 0]).reshape(1, -1))[0][0]
label_7 = enc.inverse_transform(np.array([0, 0, 0, 0, 0, 0, 0, 1]).reshape(1, -1))[0][0]
target_names = [label_0, label_1, label_2, label_3, label_4, label_5, label_6, label_7]

In [15]:
X_train_speaker = np.array(X_train_speaker)
X_val_speaker = np.array(X_val_speaker)
X_test_speaker = np.array(X_test_speaker)

In [16]:
X_train_speaker = X_train_speaker.reshape(X_train_speaker.shape[0],
                                          X_train_speaker.shape[1],
                                          X_train_speaker.shape[2],
                                          1)
X_val_speaker = X_val_speaker.reshape(X_val_speaker.shape[0],
                                      X_val_speaker.shape[1],
                                      X_val_speaker.shape[2],
                                      1)
X_test_speaker = X_test_speaker.reshape(X_test_speaker.shape[0],
                                        X_test_speaker.shape[1],
                                        X_test_speaker.shape[2],
                                        1)

In [17]:
input_shape = (X_train_speaker.shape[1], X_train_speaker.shape[2], 1)

In [18]:
model = cnn_models.paper_architecture(8, input_shape=input_shape)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 63, 56, 32)        544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 27, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 12, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)        

In [28]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val_speaker, y_val_speaker))

Train on 19008 samples, validate on 4752 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
19008/19008 [==============================] - 142s 7ms/sample - loss: 1.1053 - acc: 0.6003 - val_loss: 0.7498 - val_acc: 0.7193
Epoch 2/10
19008/19008 [==============================] - 146s 8ms/sample - loss: 0.6162 - acc: 0.7686 - val_loss: 0.4434 - val_acc: 0.8251
Epoch 3/10
19008/19008 [==============================] - 134s 7ms/sample - loss: 0.4357 - acc: 0.8363 - val_loss: 0.3235 - val_acc: 0.8832
Epoch 4/10
19008/19008 [==============================] - 130s 7ms/sample - loss: 0.3481 - acc: 0.8666 - val_loss: 0.2478 - val_acc: 0.9139
Epoch 5/10
19008/19008 [==============================] - 130s 7ms/sample - loss: 0.2947 - acc: 0.8868 - val_loss: 0.2349 - val_acc: 0.9127
Epoch 6/10
19008/19008 [==============================] - 130s 7ms/sample - loss: 0.2407 - acc: 0.9058 - val_loss: 0.1947 - val_acc: 0.9144
Epoch 7/10
19008/19008 [==============================] - 130s 

In [29]:
Y_val_nn = np.argmax(y_val_speaker, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_val_speaker)
print(classification_report(Y_val_nn, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         ale       0.96      0.94      0.95       138
      alinda       0.93      0.95      0.94       144
        gian       0.89      0.91      0.90       162
     jackson       0.99      0.99      0.99      1040
      khaled       0.99      0.91      0.95       172
     nicolas       1.00      0.97      0.99      1049
        theo       0.93      0.81      0.86      1037
    yweweler       0.80      0.94      0.87      1010

    accuracy                           0.93      4752
   macro avg       0.94      0.93      0.93      4752
weighted avg       0.93      0.93      0.93      4752



### Batch_normalization = True

In [30]:
model = cnn_models.paper_architecture(8, input_shape=input_shape, batch_normalisation=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 63, 56, 32)        544       
_________________________________________________________________
batch_normalization_v1_4 (Ba (None, 63, 56, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 27, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 12, 64)        32832     
_________________________________________________________________
batch_normalization_v1_5 (Ba (None, 14, 12, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
__________

In [31]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val_speaker, y_val_speaker))

Train on 19008 samples, validate on 4752 samples
Epoch 1/10
19008/19008 [==============================] - 145s 8ms/sample - loss: 0.8969 - acc: 0.6839 - val_loss: 0.5192 - val_acc: 0.8085
Epoch 2/10
19008/19008 [==============================] - 145s 8ms/sample - loss: 0.5761 - acc: 0.7778 - val_loss: 0.4993 - val_acc: 0.8068
Epoch 3/10
19008/19008 [==============================] - 147s 8ms/sample - loss: 0.4659 - acc: 0.8204 - val_loss: 0.4451 - val_acc: 0.8493
Epoch 4/10
19008/19008 [==============================] - 148s 8ms/sample - loss: 0.4121 - acc: 0.8445 - val_loss: 0.3313 - val_acc: 0.8813
Epoch 5/10
19008/19008 [==============================] - 146s 8ms/sample - loss: 0.3659 - acc: 0.8588 - val_loss: 0.2428 - val_acc: 0.9139
Epoch 6/10
19008/19008 [==============================] - 146s 8ms/sample - loss: 0.3373 - acc: 0.8721 - val_loss: 0.2460 - val_acc: 0.8963
Epoch 7/10
19008/19008 [==============================] - 146s 8ms/sample - loss: 0.3119 - acc: 0.8785 - val_lo

In [32]:
y_pred = model.predict_classes(X_val_speaker)
print(classification_report(Y_val_nn, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         ale       0.94      0.85      0.89       138
      alinda       0.89      0.67      0.77       144
        gian       0.78      0.86      0.82       162
     jackson       0.99      0.98      0.98      1040
      khaled       0.81      0.93      0.86       172
     nicolas       0.99      0.99      0.99      1049
        theo       0.82      0.92      0.87      1037
    yweweler       0.91      0.81      0.86      1010

    accuracy                           0.91      4752
   macro avg       0.89      0.88      0.88      4752
weighted avg       0.92      0.91      0.91      4752



### Different architecture
Let's change a bit the architecture and see if we can improve scores:

In [33]:
model = cnn_models.custom_cnn(8, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 63, 56, 32)        544       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 27, 64)        32832     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 12, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 10752)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               1376384   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 1032      
Total para

In [34]:
%%time
model.fit(X_train_speaker, y_train_speaker,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val_speaker, y_val_speaker))

Train on 19008 samples, validate on 4752 samples
Epoch 1/10
19008/19008 [==============================] - 131s 7ms/sample - loss: 1.1035 - acc: 0.6002 - val_loss: 0.6821 - val_acc: 0.7771
Epoch 2/10
19008/19008 [==============================] - 130s 7ms/sample - loss: 0.6034 - acc: 0.7754 - val_loss: 0.3892 - val_acc: 0.8434
Epoch 3/10
19008/19008 [==============================] - 134s 7ms/sample - loss: 0.4343 - acc: 0.8345 - val_loss: 0.3631 - val_acc: 0.8678
Epoch 4/10
19008/19008 [==============================] - 134s 7ms/sample - loss: 0.3276 - acc: 0.8761 - val_loss: 0.3356 - val_acc: 0.8678
Epoch 5/10
19008/19008 [==============================] - 133s 7ms/sample - loss: 0.2683 - acc: 0.8964 - val_loss: 0.2132 - val_acc: 0.9198
Epoch 6/10
19008/19008 [==============================] - 133s 7ms/sample - loss: 0.2265 - acc: 0.9109 - val_loss: 0.1985 - val_acc: 0.9211
Epoch 7/10
19008/19008 [==============================] - 135s 7ms/sample - loss: 0.2065 - acc: 0.9178 - val_lo

In [35]:
y_pred = model.predict_classes(X_val_speaker)
print(classification_report(Y_val_nn, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         ale       0.99      0.91      0.95       138
      alinda       0.93      0.96      0.95       144
        gian       0.98      0.87      0.92       162
     jackson       0.98      1.00      0.99      1040
      khaled       0.96      0.88      0.92       172
     nicolas       0.99      0.99      0.99      1049
        theo       0.98      0.74      0.84      1037
    yweweler       0.78      0.99      0.87      1010

    accuracy                           0.93      4752
   macro avg       0.95      0.92      0.93      4752
weighted avg       0.94      0.93      0.92      4752



### Digits

In [36]:
X_train_digit_nn = np.array(X_train_digit)
X_val_digit_nn = np.array(X_val_digit)
X_test_digit_nn = np.array(X_test_digit)

In [37]:
X_train_digit_nn = X_train_digit_nn.reshape(X_train_digit_nn.shape[0], X_train_digit_nn.shape[1], X_train_digit_nn.shape[2], 1)
X_val_digit_nn = X_val_digit_nn.reshape(X_val_digit_nn.shape[0], X_val_digit_nn.shape[1], X_val_digit_nn.shape[2], 1)
X_test_digit_nn = X_test_digit_nn.reshape(X_test_digit_nn.shape[0], X_test_digit_nn.shape[1], X_test_digit_nn.shape[2], 1)
y_train_digit_nn = tf.keras.utils.to_categorical(y_train_digit, 10)
y_test_digit_nn = tf.keras.utils.to_categorical(y_test_digit, 10)

In [38]:
y_val_digit_nn = tf.keras.utils.to_categorical(y_val_digit, 10)

In [39]:
input_shape = (X_train_digit_nn.shape[1], X_train_digit_nn.shape[2], 1)

#### Paper

In [40]:
model = cnn_models.paper_architecture(10, input_shape=input_shape, batch_normalisation=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
batch_normalization_v1_8 (Ba (None, 63, 56, 32)        128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 30, 27, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 12, 64)        32832     
_________________________________________________________________
batch_normalization_v1_9 (Ba (None, 14, 12, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 5, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1920)              0         
__________

In [41]:
%%time
model.fit(X_train_digit_nn, y_train_digit_nn,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val_digit_nn, y_val_digit_nn))

Train on 18480 samples, validate on 4620 samples
Epoch 1/10
18480/18480 [==============================] - 148s 8ms/sample - loss: 1.5950 - acc: 0.4432 - val_loss: 1.2522 - val_acc: 0.5530
Epoch 2/10
18480/18480 [==============================] - 143s 8ms/sample - loss: 1.1149 - acc: 0.6149 - val_loss: 0.7899 - val_acc: 0.7519
Epoch 3/10
18480/18480 [==============================] - 144s 8ms/sample - loss: 0.9122 - acc: 0.6918 - val_loss: 0.8882 - val_acc: 0.7058
Epoch 4/10
18480/18480 [==============================] - 144s 8ms/sample - loss: 0.8336 - acc: 0.7214 - val_loss: 0.7434 - val_acc: 0.7500
Epoch 5/10
18480/18480 [==============================] - 142s 8ms/sample - loss: 0.7698 - acc: 0.7439 - val_loss: 0.5994 - val_acc: 0.8035
Epoch 6/10
18480/18480 [==============================] - 146s 8ms/sample - loss: 0.7486 - acc: 0.7525 - val_loss: 0.8507 - val_acc: 0.7182
Epoch 7/10
18480/18480 [==============================] - 142s 8ms/sample - loss: 0.7086 - acc: 0.7602 - val_lo

In [42]:
Y_val_nn = np.argmax(y_val_digit_nn, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_val_digit_nn)
print(classification_report(Y_val_nn, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       486
           1       0.93      0.77      0.84       441
           2       0.96      0.67      0.79       468
           3       0.88      0.78      0.83       481
           4       0.88      0.84      0.86       467
           5       0.50      0.96      0.66       483
           6       0.63      0.89      0.74       418
           7       0.93      0.75      0.83       462
           8       0.98      0.63      0.77       470
           9       0.91      0.81      0.86       444

    accuracy                           0.80      4620
   macro avg       0.85      0.80      0.81      4620
weighted avg       0.86      0.80      0.81      4620



#### Custom

In [43]:
model = cnn_models.custom_cnn(10, input_shape=input_shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 63, 56, 32)        544       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 27, 64)        32832     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 12, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 10752)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               1376384   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1290      
Total para

In [44]:
%%time
model.fit(X_train_digit_nn, y_train_digit_nn,
          batch_size=32,
          epochs=10,
          verbose=1,
          callbacks=[callback],
          validation_data=(X_val_digit_nn, y_val_digit_nn))

Train on 18480 samples, validate on 4620 samples
Epoch 1/10
18480/18480 [==============================] - 173s 9ms/sample - loss: 1.5401 - acc: 0.4494 - val_loss: 1.1111 - val_acc: 0.6065
Epoch 2/10
18480/18480 [==============================] - 172s 9ms/sample - loss: 1.0322 - acc: 0.6351 - val_loss: 0.8405 - val_acc: 0.7245
Epoch 3/10
18480/18480 [==============================] - 161s 9ms/sample - loss: 0.8454 - acc: 0.7031 - val_loss: 0.7403 - val_acc: 0.7437
Epoch 4/10
18480/18480 [==============================] - 153s 8ms/sample - loss: 0.7291 - acc: 0.7432 - val_loss: 0.6913 - val_acc: 0.7641
Epoch 5/10
18480/18480 [==============================] - 156s 8ms/sample - loss: 0.6545 - acc: 0.7713 - val_loss: 0.5594 - val_acc: 0.8026
Epoch 6/10
18480/18480 [==============================] - 153s 8ms/sample - loss: 0.6030 - acc: 0.7891 - val_loss: 0.5868 - val_acc: 0.7950
Epoch 7/10
18480/18480 [==============================] - 161s 9ms/sample - loss: 0.5485 - acc: 0.8086 - val_lo

In [45]:
y_pred = model.predict_classes(X_val_digit_nn)
print(classification_report(Y_val_nn, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93       486
           1       0.83      0.86      0.85       441
           2       0.93      0.78      0.85       468
           3       0.70      0.91      0.79       481
           4       0.80      0.91      0.85       467
           5       0.96      0.83      0.89       483
           6       0.70      0.91      0.79       418
           7       0.98      0.77      0.86       462
           8       0.93      0.84      0.88       470
           9       0.93      0.86      0.89       444

    accuracy                           0.85      4620
   macro avg       0.87      0.86      0.86      4620
weighted avg       0.87      0.85      0.86      4620



# Test model 

In [56]:
import sounddevice as sd
import subprocess

import time
import librosa

import IPython.display as ipd

import os
from scipy.io import wavfile as wav

In [ ]:
def pad_zeros_single_rec(rec, max_y):
    rec = np.array(rec)
    diff_in_y = max_y - rec.shape[0]
    if diff_in_y > 0:
        half_diff = int(diff_in_y/2)
        remaining_diff = diff_in_y-half_diff
        v = np.pad(rec, (half_diff, remaining_diff), 'constant', constant_values=0)
        return v
    else:
        return rec

In [ ]:
def create_recording(duration, rec_rate, name = "test.wav", output_dir = "test/"):
    print("Ready in 3...", end = "")
    time.sleep(1)
    print("2...", end = "")
    time.sleep(1)
    print("1...")
    time.sleep(1)
    print("Go.")
    rec = sd.rec(int(duration * rec_rate), samplerate=rec_rate, channels=1, blocking=True)
    print("Playing the recording.")
    sd.play(rec, rec_rate)

    # after hearing the recording, decide whether to record it again or continue to next number
    # if you type anything, record again
    # if you press enter, save current recording & go to next number
    ok = input("OK?")
    if ok == "":
        librosa.output.write_wav(output_dir+name, rec, rec_rate)
        return rec
    ipd.clear_output(wait=True)
    create_recording(duration, rec_rate)

In [ ]:
def trim_audio(file, input_dir="test/", output_dir="test/", db=-48):

    if not os.path.isdir(input_dir):
        print(f"There should be an input \"{input_dir}\" directory.")
        sys.exit(0)
    
    # create output directory if not there yet
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
        
    temp1 = output_dir+"temp1.wav"
    temp2 = output_dir+"temp2.wav"
    temp3 = output_dir+"temp3.wav"
 
    subprocess.run(["ffmpeg", "-y", "-i", input_dir+file, "-af", f"silenceremove=1:0:{db}dB", temp1])
    subprocess.run(["ffmpeg", "-y", "-i", temp1, "-af", "areverse", temp2])
    subprocess.run(["ffmpeg", "-y", "-i", temp2, "-af", f"silenceremove=1:0.1:{db}dB", temp3])
    subprocess.run(["ffmpeg", "-y", "-i", temp3, "-af", "areverse", output_dir+file])
    
    os.remove(temp1)
    os.remove(temp2)
    os.remove(temp3)

In [ ]:
def test_NN(nn, max_y, target_names, answer = None, duration=2, rec_rate=8000, directory = "test/", filename = "test.wav"):
    create_recording(duration, rec_rate, filename, directory)   
    ipd.clear_output()
    trim_audio(filename, directory, directory)
    # _, rec = wav.read(directory + "/" + filename)
    rec, _ = librosa.core.load(directory + "/" + filename, sr = rec_rate)
    rec = pad_zeros_single_rec(rec, max_y)
    # sd.play(rec, rec_rate)
    rec = data_preparation.compute_spectrogram(rec, normalize=True)
    rec = rec[np.newaxis,:,:,np.newaxis]
    preds = nn.predict_classes(rec)
    print("Model prediction: {}".format(target_names[preds[0]]))
    if answer is not None:
        print(f"Correct answer {answer}")
    return preds

In [ ]:
max_y = len(data_augm_pad_recordings[1])

In [ ]:
pred = test_NN(model, max_y, target_names, answer = "gian")

# TO DO:
- [x] Set random seed
- [x] Use only original recordings in test set of augmented scenario
- [x] Use proper validation set for picking best models and params
- [x] Data augmentation also for digit recognition
- [ ] Evaluate each best model on test set, after training it on x_train + x_test